<a href="https://colab.research.google.com/github/tmdoi/small-Japanese-LLM-compare/blob/main/tokyotech_llm_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

情報源
[
tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.5](https://huggingface.co/tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.5)

In [ ]:
!nvidia-smi

In [ ]:
%pip -q install --upgrade pip
%pip -q install transformers accelerate bitsandbytes huggingface_hub


In [ ]:
import os

# vLLM が要求するマルチプロセス方式を事前に明示
os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"

# 単一GPUを明示（Colab L4想定）
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# まれな相性問題の回避に有効な場合あり（必要時のみ有効化）
# os.environ["VLLM_USE_TRITON_FLASH_ATTENTION"] = "0"

print("環境変数を設定しました。")


In [ ]:
from huggingface_hub import login
login()  # 対話プロンプトに従って hf_ から始まるトークンを貼り付け


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

MODEL = "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.5"

tok = AutoTokenizer.from_pretrained(MODEL, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    device_map="auto",
    torch_dtype=torch.float16,   # T4 は bf16 非対応
    load_in_4bit=True,           # ★ ここがポイント（bitsandbytes 4bit）
    bnb_4bit_compute_dtype=torch.float16,
)

prompt = tok.apply_chat_template(
    [
        {"role": "system", "content": "丁寧・簡潔・明晰に回答してください。"},
        {"role": "user", "content": "東京の紅葉した公園で…（略）…物語を書いてください。"}
    ],
    tokenize=False, add_generation_prompt=True
)

inputs = tok(prompt, return_tensors="pt").to(model.device)
streamer = TextStreamer(tok, skip_prompt=True, skip_special_tokens=True)

_ = model.generate(
    **inputs,
    max_new_tokens=512,
    temperature=0.6, top_p=0.9,
    do_sample=True,
    streamer=streamer
)
